### Veri Seti Bilgisi

IMDB veri seti, doğal dil işleme için yani ikili duygu sınıflandırması için 50 bin film incelemesi içerir. Veri seti, duygusal analizi gerçekleştirmek için iki sütun içerir - inceleme ve duygu.

### Amaç
IMDB yorumlarındaki olumlu ve olumsuz duyguları doğru şekilde sınıflandırmak.

### Genel Bakis
Bu not defterinde, makine öğrenimi ve derin öğrenme algoritmalarını kullanarak veri kümesinde temizlik, keşifsel veri analizi ve öngörücü modelleme gerçekleştiriyoruz. Makine öğrenimi için Lineer Regresyon, Random Forest Classifier ve Lojistik Regresyon kullanıyoruz. Maksimum doğruluk değerimiz Lojistik Regresyon için yaklaşık %89'dır.

### Referanslar
1. https://www.kaggle.com/madz2000/sentiment-analysis-cleaning-eda-bert-88-acc/output
2. https://www.kaggle.com/aditya6040/7-models-on-imdb-dataset-best-score-88-2
3. https://www.kaggle.com/faressayah/sentiment-model-with-tensorflow-transformers
4. https://www.kaggle.com/sravyaysk/imdb-movie-review-classification-using-tensorflow
5. https://www.kaggle.com/colearninglounge/nlp-data-preprocessing-and-cleaning
6. https://huggingface.co/transformers/master/custom_datasets.html

# **1. Kütüphanelerin İçe Aktarılması**


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud,STOPWORDS
from bs4 import BeautifulSoup
import re,string,unicodedata

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

import transformers
import tokenizers
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# **2. Veri Çıkarma ve Temizleme**


In [ ]:
if not os.path.exists("IMDB-Dataset.csv"):
    print("Dosya bulunamadı, indiriliyor...")
    !wget https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv
else:
    print("Dosya zaten mevcut, yeniden indirilmedi.")

Dosya bulunamadı, indiriliyor...
--2024-12-04 10:03:15--  https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/plain]
Saving to: ‘IMDB-Dataset.csv’

IMDB-Dataset.csv    100%[===================>]  63.14M   131MB/s    in 0.5s    

2024-12-04 10:03:17 (131 MB/s) - ‘IMDB-Dataset.csv’ saved [66212309/66212309]



Aşağıdaki hücre, .csv verilerini okumak ve ilk 5 girdiyi görselleştirmek içindir.


In [ ]:
data=pd.read_csv('IMDB-Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Aşağıdaki hücre, açıklayıcı istatistikleri göstermek için kullanılır; bu istatistikler arasında benzersiz inceleme sayısı, mod ve her bir sütunun (review ve sentiment) modunun frekansı yer alır.


In [ ]:
data.describe() #descriptive statistics

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


Aşağıdaki hücreler, null değerleri analiz etmek için kullanılır. Görüldüğü üzere null değer sayısı 0'dır.


In [ ]:
null_values = data.isnull().sum()

In [ ]:
null_values.index[0]

'review'

In [ ]:
print('{} için {} eksik değer ve {} için {} eksik değer bulunmaktadır.'.format(null_values[0], null_values.index[0], null_values[1], null_values.index[1]))


0 için review eksik değer ve 0 için sentiment eksik değer bulunmaktadır.


Aşağıdaki hücreler, tekrar eden incelemelerle ilgilenmek ve tekrar edenleri kaldırmak içindir.


In [ ]:
num_duplicates = data.duplicated().sum()
print('Veri kümesinde {} adet tekrar eden inceleme bulunmaktadır.'.format(num_duplicates))


Veri kümesinde 418 adet tekrar eden inceleme bulunmaktadır.


In [ ]:
# tekrar eden incelemeleri görüntüle

review = data['review']
duplicated_review = data[review.isin(review[review.duplicated()])].sort_values("review")
duplicated_review.head()

,review,sentiment
34058,"""Go Fish"" garnered Rose Troche rightly or wron...",negative
47467,"""Go Fish"" garnered Rose Troche rightly or wron...",negative
29956,"""Three"" is a seriously dumb shipwreck movie. M...",negative
31488,"""Three"" is a seriously dumb shipwreck movie. M...",negative
47527,"""Witchery"" might just be the most incoherent a...",negative


In [ ]:
#tekrar eden incelemeleri çıkar
data.drop_duplicates(inplace = True)

In [ ]:
print('Tekrar edenler kaldırıldıktan sonra veri kümesi {} satır ve {} sütun içermektedir.'.format(data.shape[0], data.shape[1]))


Tekrar edenler kaldırıldıktan sonra veri kümesi 49582 satır ve 2 sütun içermektedir.


Aşağıdaki hücre, bağlama anlam katmayan "a", "an", "the" gibi durak kelimelerle ilgilenir. Bu kelimeler, duygu analizi sırasında kaldırılacaktır. WordNetLemmatizer fonksiyonu, çoğul kelimeleri tekil hale getirir, böylece makine metni daha iyi anlayabilir.


In [ ]:
stop = stopwords.words('english')
wl = WordNetLemmatizer()

Aşağıdaki hücre, İngilizcede yaygın olarak kullanılan kısaltmalar için bir eşleme sağlar, böylece veri kümesinde hiçbir kısaltma bulunmaz.


In [ ]:
mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot",
           "'cause": "because", "could've": "could have", "couldn't": "could not",
           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
           "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will",
           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
           "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
           "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have",
           "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
           "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
           "might've": "might have","mightn't": "might not","mightn't've": "might not have",
           "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
           "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
           "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
           "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
           "she's": "she is", "should've": "should have", "shouldn't": "should not",
           "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
           "that'd": "that would", "that'd've": "that would have", "that's": "that is",
           "there'd": "there would", "there'd've": "there would have", "there's": "there is",
           "here's": "here is","they'd": "they would", "they'd've": "they would have",
           "they'll": "they will", "they'll've": "they will have", "they're": "they are",
           "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
           "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
           "we're": "we are", "we've": "we have", "weren't": "were not",
           "what'll": "what will", "what'll've": "what will have","what're": "what are",
           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
           "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
           "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
           "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
           "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
           "y'all're": "you all are","y'all've": "you all have","you'd": "you would",
           "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
           "you're": "you are", "you've": "you have" }

Aşağıdaki hücre, metni dil modeli için anlamsız olan tüm olası varlıklardan (html etiketleri, emojiler, kısaltmalar ve çoğullar gibi) temizlemek için bir fonksiyon içerir.



In [ ]:
# Veriyi temizleme fonksiyonu
def clean_text(text, lemmatize=True):
    soup = BeautifulSoup(text, "html.parser")  # HTML etiketlerini kaldırır
    text = soup.get_text()
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])  # Kısaltmaları ve kontraksiyonları açar
    emoji_clean = re.compile("["
                             u"\U0001F600-\U0001F64F"  # ifadeler (emoticons)
                             u"\U0001F300-\U0001F5FF"  # semboller ve piktogramlar
                             u"\U0001F680-\U0001F6FF"  # ulaşım ve harita sembolleri
                             u"\U0001F1E0-\U0001F1FF"  # bayraklar (iOS)
                             u"\U00002702-\U000027B0"
                             u"\U000024C2-\U0001F251"
                             "]+", flags=re.UNICODE)
    text = emoji_clean.sub(r'', text)  # Emojileri kaldırır
    text = re.sub(r'\.(?=\S)', '. ', text)  # Noktadan sonra boşluk ekler
    text = re.sub(r'http\S+', '', text)  # URL'leri kaldırır
    text = "".join([word.lower() for word in text if word not in string.punctuation])  # Noktalama işaretlerini kaldırır
    # tokens = re.split('\W+', text)  # Token oluşturur
    if lemmatize:
        text = " ".join([wl.lemmatize(word) for word in text.split() if word not in stop and word.isalpha()])  # Lemmatizasyon yapar
    else:
        text = " ".join([word for word in text.split() if word not in stop and word.isalpha()])  # Lemmatizasyon yapmadan kelimeleri işler
    return text


Veri kümesinin bir kopyasını oluştur.


In [ ]:
data_copy = data.copy()

Temizleme fonksiyonunu uygula.


In [ ]:
data['review']=data['review'].apply(clean_text,lemmatize = True)

In [ ]:
# Hedef değişkeni sayısal etiketlere dönüştürme
data.sentiment = [ 1 if each == "positive" else 0 for each in data.sentiment]

In [ ]:
data.head()

# **3. Keşifsel Veri Analizi**


Bu bölümde Keşifsel Veri Analizi (Exploratory Data Analysis - EDA) yapacağız.


Aşağıdaki hücre, olumlu ve olumsuz incelemelerin sayısını görselleştirmek için kullanılır.


In [ ]:
### Sayım Grafiği

sns.set(style = "whitegrid" , font_scale = 1.2)
sns.countplot(data.sentiment,palette = ['green','red'],order = [1,0])
plt.xticks(ticks = np.arange(2),labels = ['positive','negative'])
plt.title('Target count for IMBD reviews')
plt.show()

In [ ]:
print('Pozitif review sayısı', (round(data['sentiment'].value_counts()[0])),'ve datasetin ', round(data['sentiment'].value_counts()[0]/len(data) * 100,2), '% sini oluşturuyor')
print('Negatif review sayısı', (round(data['sentiment'].value_counts()[1])),'ve datasetin',round(data['sentiment'].value_counts()[1]/len(data) * 100,2), '% sini oluşturuyor')

Bu bölümde, hem olumlu hem de olumsuz yorumlarda en sık kullanılan kelimelerin daha büyük boyutlarla gösterildiği görseller oluşturmak için **wordcloud** kütüphanesini kullandım.

In [ ]:
# Olumlu İncelemeler için Word Cloud
positive_data = data[data.sentiment == 1]['review']
positive_data_string = ' '.join(positive_data)
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width=1200, height=600,background_color="white").generate(positive_data_string)
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.title('# Olumlu İncelemeler için Word Cloud',fontsize = 20)
plt.show()

In [ ]:
# Olumsuz İncelemeler için Word Cloud

negative_data = data[data.sentiment == 0]['review']
negative_data_string = ' '.join(negative_data)
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width=1200, height=600,background_color="white").generate(negative_data_string)
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.title(' Olumsuz İncelemeler için Word Cloud',fontsize = 20)
plt.show()

Şimdi olumlu ve olumsuz incelemelerin uzunlukları karşılaştırılacak.


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
text_len = positive_data.str.len()
ax1.hist(text_len, color='green')
ax1.set_title('Olumlu İncelemeler')
ax1.set_xlabel('Karakter Sayısı')
ax1.set_ylabel('Sayı')
text_len = negative_data.str.len()
ax2.hist(text_len, color='red')
ax2.set_title('Olumsuz İncelemeler')
ax2.set_xlabel('Karakter Sayısı')
ax2.set_ylabel('Sayı')
fig.suptitle('Metinlerdeki Karakter Sayısı')
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

text_len = positive_data.str.split().map(lambda x: len(x))
ax1.hist(text_len, color='green')
ax1.set_title('Olumlu İncelemeler')
ax1.set_xlabel('Kelime Sayısı')
ax1.set_ylabel('Sayı')
text_len = negative_data.str.split().map(lambda x: len(x))
ax2.hist(text_len, color='red')
ax2.set_title('Olumsuz İncelemeler')
ax2.set_xlabel('Kelime Sayısı')
ax2.set_ylabel('Sayı')
fig.suptitle('Metinlerdeki Kelime Sayısı')
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

word = positive_data.str.split().apply(lambda x: len(x))
sns.histplot(word, ax=ax1, color='green', kde=True)  # histplot kullanılarak güncellendi
ax1.set_title('Olumlu İncelemeler')
ax1.set_xlabel('İnceleme Başına Kelime Sayısı')

word = negative_data.str.split().apply(lambda x: len(x))
sns.histplot(word, ax=ax2, color='red', kde=True)  # histplot kullanılarak güncellendi
ax2.set_title('Olumsuz İncelemeler')
ax2.set_xlabel('İnceleme Başına Kelime Sayısı')

fig.suptitle('İnceleme Başına Kelime Sayısının Dağılımı')
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

word = positive_data.str.split().apply(lambda x: [len(i) for i in x])
sns.histplot(word.map(lambda x: np.mean(x)), ax=ax1, color='green', kde=True)  # distplot yerine histplot kullanıldı
ax1.set_title('Olumlu İncelemeler')
ax1.set_xlabel('İnceleme Başına Ortalama Kelime Uzunluğu')

word = negative_data.str.split().apply(lambda x: [len(i) for i in x])
sns.histplot(word.map(lambda x: np.mean(x)), ax=ax2, color='red', kde=True)  # distplot yerine histplot kullanıldı
ax2.set_title('Olumsuz İncelemeler')
ax2.set_xlabel('İnceleme Başına Ortalama Kelime Uzunluğu')

fig.suptitle('Her İncelemedeki Ortalama Kelime Uzunluğunun Dağılımı')
plt.show()


Görüldüğü üzere, olumlu incelemeler genellikle olumsuz incelemelere kıyasla daha kısa olma eğilimindedir. Ayrıca, genel olarak daha kısa kelimeler içerirler.


Aşağıdaki hücreler, veri kümesinden bir corpus (metin kümesi) oluşturmak içindir. Bu, veri kümesindeki kelime frekanslarını analiz etmek için yapılır.


In [ ]:
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(data.review)
corpus[:5]

In [ ]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = pd.DataFrame(most_common,columns = ['corpus','countv'])
most_common

In [ ]:
most_common = most_common.sort_values('countv')

In [ ]:
plt.figure(figsize =(10,10))
plt.yticks(range(len(most_common)), list(most_common.corpus))
plt.barh(range(len(most_common)), list(most_common.countv),align='center',color = 'blue')
plt.title('Datasette en çok kullanılan kelimeler')
plt.show()


Şaşırtıcı olmayan bir şekilde, metin kümesindeki en yaygın kelimeler "movie" ve "film"dir.


N-gramlar, geleneksel doğal dil işleme (NLP) için çok önemlidir. Bir unigram (n=1 ngram) tek bir kelimeden oluşurken, bir bigram (n=2) ardışık 2 kelimeden oluşur, vb. Aşağıdaki hücreler, n-gramları elde etmek ve farklı n-gramlar için olumlu ve olumsuz incelemeler arasında bir karşılaştırma yapmak için kullanılmıştır.


In [ ]:
def get_ngrams(review, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(review)  # Belirtilen n-gram aralığında kelimeleri vektörize eder
    bag_of_words = vec.transform(review)  # CountVectorizer'ın sparse matrisini oluşturur
    sum_words = bag_of_words.sum(axis=0)  # Toplam kelime sayısını hesaplar
    sum_words = np.array(sum_words)[0].tolist()  # Listeye dönüştürür
    words_freq = [(word, sum_words[idx]) for word, idx in vec.vocabulary_.items()]  # Kelimelerin frekansını ve konumunu alır
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)  # Kelime frekansına göre azalan şekilde sıralar
    return words_freq[:n]  # İlk n kelimeyi döndürür


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

# Olumlu incelemeler için unigram analizi
uni_positive = get_ngrams(positive_data, 20, 1)
uni_positive = dict(uni_positive)
temp = pd.DataFrame(list(uni_positive.items()), columns=["En Yaygın Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın Kelimeler", orient='h', ax=ax1)
ax1.set_title('Olumlu İncelemeler')

# Olumsuz incelemeler için unigram analizi
uni_negative = get_ngrams(negative_data, 20, 1)
uni_negative = dict(uni_negative)
temp = pd.DataFrame(list(uni_negative.items()), columns=["En Yaygın Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın Kelimeler", orient='h', ax=ax2)
ax2.set_title('Olumsuz İncelemeler')

fig.suptitle('Olumlu ve Olumsuz İncelemeler için Unigram Analizi')
plt.show()


Görüldüğü üzere, olumlu ve olumsuz incelemeler sırasıyla daha sık olumlu ve olumsuz kelimeler içermektedir.


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

# Olumlu incelemeler için bigram analizi
bi_positive = get_ngrams(positive_data, 20, 2)
bi_positive = dict(bi_positive)
temp = pd.DataFrame(list(bi_positive.items()), columns=["En Yaygın İkili Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın İkili Kelimeler", orient='h', ax=ax1)
ax1.set_title('Olumlu İncelemeler')

# Olumsuz incelemeler için bigram analizi
bi_negative = get_ngrams(negative_data, 20, 2)
bi_negative = dict(bi_negative)
temp = pd.DataFrame(list(bi_negative.items()), columns=["En Yaygın İkili Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın İkili Kelimeler", orient='h', ax=ax2)
ax2.set_title('Olumsuz İncelemeler')

fig.suptitle('Olumlu ve Olumsuz İncelemeler için Bigram Analizi')
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

# Olumlu incelemeler için trigram analizi
tri_positive = get_ngrams(positive_data, 20, 3)
tri_positive = dict(tri_positive)
temp = pd.DataFrame(list(tri_positive.items()), columns=["En Yaygın Üçlü Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın Üçlü Kelimeler", orient='h', ax=ax1)
ax1.set_title('Olumlu İncelemeler')

# Olumsuz incelemeler için trigram analizi
tri_negative = get_ngrams(negative_data, 20, 3)
tri_negative = dict(tri_negative)
temp = pd.DataFrame(list(tri_negative.items()), columns=["En Yaygın Üçlü Kelimeler", 'Sıklık'])
sns.barplot(data=temp, x="Sıklık", y="En Yaygın Üçlü Kelimeler", orient='h', ax=ax2)
ax2.set_title('Olumsuz İncelemeler')

fig.suptitle('Olumlu ve Olumsuz İncelemeler için Trigram Analizi')
plt.show()


N arttıkça, en yaygın kullanılan n-gramlar arasındaki fark daha belirgin hale gelir.


# **4. Makine Öğrenmesi Kullanarak Tahminsel Modelleme**


In [ ]:
# Eğitim ve test veri setlerine ayırma
train, test= train_test_split(data, test_size=0.2, random_state=42)
Xtrain, ytrain = train['review'], train['sentiment']
Xtest, ytest = test['review'], test['sentiment']

In [ ]:
# Verilerin Vektörleştirilmesi

tfidf_vect = TfidfVectorizer()
Xtrain_tfidf = tfidf_vect.fit_transform(Xtrain)
Xtest_tfidf = tfidf_vect.transform(Xtest)


count_vect = CountVectorizer()
Xtrain_count = count_vect.fit_transform(Xtrain)
Xtest_count = count_vect.transform(Xtest)

### Lineer Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

lr = LinearRegression()
lr.fit(Xtrain_tfidf, ytrain)
p1 = lr.predict(Xtest_tfidf)
p1_rounded = [1 if pred >= 0.5 else 0 for pred in p1]
s1 = accuracy_score(ytest, p1_rounded)
print("Linear Regression Accuracy :", "{:.2f}%".format(100 * s1))
ConfusionMatrixDisplay.from_predictions(ytest, p1_rounded, cmap='Blues')
plt.grid(False)
plt.show()


### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(Xtrain_tfidf, ytrain)
p2 = random_forest.predict(Xtest_tfidf)
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
s2 = accuracy_score(ytest, p2)
print("Random Forest Classifier Accuracy :", "{:.2f}%".format(100 * s2))
ConfusionMatrixDisplay.from_estimator(random_forest, Xtest_tfidf, ytest, cmap='Blues')
plt.grid(False)
plt.show()


### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

lr = LogisticRegression()
lr.fit(Xtrain_tfidf,ytrain)
p3=lr.predict(Xtest_tfidf)
s3=accuracy_score(ytest,p3)
print("Logistic Regression Accuracy :", "{:.2f}%".format(100*s3))
ConfusionMatrixDisplay.from_estimator(lr, Xtest_tfidf, ytest,cmap = 'Blues')
plt.grid(False)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd


validation_accuracy = accuracy_score(ytest, p3)
print("Lojistik Regresyon Doğruluğu: {:.2f}%".format(validation_accuracy * 100))
report = classification_report(ytest, p3, output_dict=True)
report_df = pd.DataFrame(report).transpose()

print("Lojistik Regresyon Metrikleri:\n")
print(report_df)


### Karşılaştırma

In [ ]:
import matplotlib.pyplot as plt

model_names = ["Lin Reg.", "Ran For.", "Log Reg."]
accuracies = [s1, s2, s3]


print("Model Accuracy Karşılaştırma:")
for name, acc in zip(model_names, accuracies):
    print(f"{name}: {acc:.2f}")


plt.figure(figsize=(5,4))
plt.bar(model_names, accuracies, color='darkblue')
plt.xlabel("Modeller")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Karşılaştırma")
plt.ylim(0, 1)
plt.yticks([i / 20 for i in range(0, 20, 1)])
plt.show()


